In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from models import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
def evaluateCorInfoMaxV3(model, loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                         neural_lr_decay_multiplier,
                         neural_dynamic_iterations, device, printing = True):
    # Evaluate the model on a dataloader with T steps for the dynamics
    #model.eval()
    correct=0
    phase = 'Train' if loader.dataset.train else 'Test'
    
    for x, y in loader:
        x = x.view(x.size(0),-1).to(device).T
        #x = 2*x - 1
        y = y.to(device)
        
        neurons = model.fast_forward(x)
        
        # dynamics for T time steps
        neurons = model.run_neural_dynamics(x, y_one_hot, neurons, neural_lr_start, neural_lr_stop, 
                                            neural_lr_rule,
                                            neural_lr_decay_multiplier, neural_dynamic_iterations, 0, "test")
        pred = torch.argmax(neurons[-1], dim=0).squeeze()  # in this case prediction is done directly on the last (output) layer of neurons
        correct += (y == pred).sum().item()

    acc = correct/len(loader.dataset) 
    if printing:
        print(phase+' accuracy :\t', acc)   
    return acc

# def evaluateCorInfoMaxV4(model, loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
#                          neural_lr_decay_multiplier,
#                          neural_dynamic_iterations, device, printing = True):
#     # Evaluate the model on a dataloader with T steps for the dynamics
#     #model.eval()
#     correct=0
#     phase = 'Train' if loader.dataset.train else 'Test'
    
#     for x, y in loader:
#         x = x.view(x.size(0),-1).to(device).T
#         #x = 2*x - 1
#         y = y.to(device)
        
#         neurons = model.fast_forward(x)
        
# #         # dynamics for T time steps
# #         neurons = model.run_neural_dynamics(x, y_one_hot, neurons, neural_lr_start, neural_lr_stop, 
# #                                             neural_lr_rule,
# #                                             neural_lr_decay_multiplier, neural_dynamic_iterations, 0, "test")
#         pred = torch.argmax(neurons[-1], dim=0).squeeze()  # in this case prediction is done directly on the last (output) layer of neurons
#         correct += (y == pred).sum().item()

#     acc = correct/len(loader.dataset) 
#     if printing:
#         print(phase+' accuracy :\t', acc)   
#     return acc

def evaluateCorInfoMaxV4(model, loader, device, printing = True):
    # Evaluate the model on a dataloader with T steps for the dynamics
    #model.eval()
    correct=0
    phase = 'Train' if loader.dataset.train else 'Test'
    
    for x, y in loader:
        x = activation_inverse(x.view(x.size(0),-1).T, "sigmoid").to(device)
#         x = x.view(x.size(0),-1).T.to(device)
        y = y.to(device)
        
        neurons = model.fast_forward(x)
        
#         # dynamics for T time steps
#         neurons = model.run_neural_dynamics(x, y_one_hot, neurons, neural_lr_start, neural_lr_stop, 
#                                             neural_lr_rule,
#                                             neural_lr_decay_multiplier, neural_dynamic_iterations, 0, "test")
        pred = torch.argmax(neurons[-1], dim=0).squeeze()  # in this case prediction is done directly on the last (output) layer of neurons
        correct += (y == pred).sum().item()

    acc = correct/len(loader.dataset) 
    if printing:
        print(phase+' accuracy :\t', acc)   
    return acc

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [5]:
# activation_type = "sigmoid"
# # activation = F.hardtanh
# architecture = [784, 500, 10]

# beta = 1
# lambda_ = 0.9999
# epsilon = 1#0.5
# psiv = 0.1
# one_over_epsilon = 1 / epsilon
# lr_start = {'ff' : 0.01, 'fb': 0.001, 'lat': 1e-3}
# neural_lr_start = 0.1
# neural_lr_stop = 1e-3
# neural_lr_rule = "divide_by_slow_loop_index"
# neural_lr_decay_multiplier = 0.1
# neural_dynamic_iterations = 20
# output_sparsity = True
# STlambda_lr = 0.005
# model = CorInfoMaxErrorProp( architecture = architecture, lambda_ = lambda_,
#                             epsilon = epsilon, psiv = psiv, activation_type = activation_type,
#                             output_sparsity = output_sparsity, STlambda_lr = STlambda_lr
#                               )

In [6]:
activation_type = "sigmoid"
# activation = F.hardtanh
architecture = [784, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T

beta = 1
lambda_ = 1 - 1e-5
epsilon = 1#0.5
psiv = 0.01
one_over_epsilon = 1 / epsilon
lr_start = {'ff' : 0.001, 'fb': 0.001, 'lat': 1e-3}
neural_lr_start = 0.1
neural_lr_stop = 1e-3
neural_lr_rule = "divide_by_slow_loop_index"
neural_lr_decay_multiplier = 0.1
neural_dynamic_iterations = 50
output_sparsity = True
STlambda_lr = 0.005
model = CorInfoMaxErrorProp( architecture = architecture, lambda_ = lambda_,
                            epsilon = epsilon, psiv = psiv, activation_type = activation_type,
                            output_sparsity = output_sparsity, STlambda_lr = STlambda_lr
                              )

In [7]:
activation_type = "sigmoid"
# activation = F.hardtanh
architecture = [784, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T

beta = 1
lambda_ = 1 - 1e-3
epsilon = 0.1#0.5
psiv = 0.1
one_over_epsilon = 1 / epsilon
lr_start = {'ff' : 0.005, 'fb': 0.005, 'lat': 1e-3}
neural_lr_start = 0.01
neural_lr_stop = 1e-3
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.1
neural_dynamic_iterations = 50
output_sparsity = True
STlambda_lr = 0.005
model = CorInfoMaxErrorProp( architecture = architecture, lambda_ = lambda_,
                            epsilon = epsilon, psiv = psiv, activation_type = activation_type,
                              )

In [8]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 50

for epoch_ in range(n_epochs):
    lr = {'ff' : lr_start['ff'] * (0.99)**epoch_, 'fb' : lr_start['fb'] * (0.99)**epoch_}
    for idx, (x, y) in tqdm(enumerate(train_loader) ):
        x, y = x.to(device), y.to(device)
#         x = x.view(x.size(0),-1).T.to(device)
        x = activation_inverse(x.view(x.size(0),-1).T, activation_type).to(device)
        y_one_hot = F.one_hot(y, 10).to(device).T

        _ = model.batch_step(x, y_one_hot, lr, neural_lr_start, neural_lr_stop,
                             neural_lr_rule, neural_lr_decay_multiplier,
                             neural_dynamic_iterations, beta, clip_neural_grad_updates = True, 
                             optimizer = "sgd",
                             adam_opt_params = {"beta1": 0.9, "beta2": 0.999, "eps": 1e-8})
    trn_acc = evaluateCorInfoMaxV4(model, train_loader, device = 'cuda', printing = False)
    tst_acc = evaluateCorInfoMaxV4(model, test_loader, device = 'cuda', printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))
    

3000it [01:36, 31.18it/s]
3it [00:00, 28.35it/s]

Epoch : 1, Train Accuracy : 0.15516666666666667, Test Accuracy : 0.1601


3000it [01:36, 31.02it/s]
3it [00:00, 28.54it/s]

Epoch : 2, Train Accuracy : 0.21411666666666668, Test Accuracy : 0.2194


3000it [01:35, 31.33it/s]
3it [00:00, 28.43it/s]

...

Epoch : 48, Train Accuracy : 0.5371833333333333, Test Accuracy : 0.5491


3000it [01:39, 30.06it/s]
4it [00:00, 30.94it/s]

Epoch : 49, Train Accuracy : 0.5377833333333333, Test Accuracy : 0.5522


3000it [01:39, 30.20it/s]


Epoch : 50, Train Accuracy : 0.5371333333333334, Test Accuracy : 0.5508
